In [1]:
# -*- coding: utf-8 -*-
import os
import math
from tqdm import tqdm
from collections import deque
import random
import numpy as np
import pandas as pd
import ast
import random as ran
import datetime
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from configuration import state_dim, action_dim, hidden_dim, lr, gamma, epsilon, batch_size, memory_size, max_trace_length, output_dim, num_layers, embedding_dim, em_size
from nn.two_tower_lstm import TwoTowerLSTM
import torch
import torch.nn as nn
import torch.optim as optim
# from skimage.transform import resize
# from skimage.color import rgb2gray
plt.ion()

In [2]:
DATA_STORE = "./datastore"

DIR_GAMES_ALL = os.listdir(DATA_STORE)
number_of_total_game = len(DIR_GAMES_ALL)

In [9]:
a = [1,2,3,4]
a[2:5]

[3, 4]

In [7]:
from model import SarsaLSTMAgent

# SARSA 기반 Main() 함수 구현을 통해 학습 실행
def main():
    agent = SarsaLSTMAgent(state_dim, action_dim, hidden_dim, lr, gamma, epsilon, batch_size, memory_size, max_trace_length, output_dim, num_layers, embedding_dim, em_size)
    epoch = 0
    # Train agent during 100 epoch
    while epoch < 1:
        epoch += 1
        # episode = num(goal sequence) | "We divide a soccer game into goal-scoring episodes"
        for game in [7525]:
            episode = os.listdir(DATA_STORE + f'/{game}')
            for epi in [1]:
                # load data
                s = np.load(DATA_STORE + f'/{game}/{epi}/state.npy', allow_pickle=True)
                r = np.load(DATA_STORE + f'/{game}/{epi}/reward.npy', allow_pickle=True)
                a = np.load(DATA_STORE + f'/{game}/{epi}/action.npy', allow_pickle=True)
                # Memory -> T(state_t, action_t, reward_t+1, state_t+1, action_t+1, Team, Trace_Length)
                # 이렇게 되면 한 턴만에 빼앗기는 경우, Q를 계산하지 않음.
                trace_length = 1
                for t in range(len(s)):
                    # t이 마지막인 경우, 종료
                    if sum(r[t]) == 1:
                        break
                    # t's team == t+1's team
                    if s[t][-1] == s[t+1][-1]:
                        agent.store_transition(s[t], a[t], r[t+1], s[t+1], a[t+1], s[t][-1], trace_length)
                        trace_length += 1
                    else:
                        # t의 team과 t+1의 team이 다른 경우, trace_length 초기화
                        trace_length = 0

                batch = agent.sample_batch()
                print(batch)
                # print(mi)
                # agent update(calc_q_loss)
                # agent.update_model()
                # # agent's memory init
                # agent.store_init()

        print("Epoch:{}".format(episode))


if __name__ == "__main__":
    main()

[[]]
Epoch:['1', '2', '3', '4', '5', '6', '7']


In [6]:
def train_sarsa_lstm(agent, num_episodes=1):
    for episode in range(num_episodes):
        trace = []
        
        # 하나의 에피소드에서 minibatch를 수행
        # 조건 1. t/t+1은 모두 같은 팀이 수행한 행동일 것 
        # 조건 2. trace length를 고려해서 batch를 잘라야 함
        # 조건 3. 하나의 state(t)는 sequence를 담고 있음. 
        for event in episode_data:
            state, action, reward, next_state, done = event
            next_action = agent.select_action(next_state)
            
            trace.append((state, action, reward, next_state, next_action, done))
            if len(trace) == agent.trace_length or done:
                for t in trace:
                    agent.store_transition(*t)
                agent.update_model()
                trace = []
                
            total_reward += reward
            
            if done:
                break
        
        print(f"Episode {episode + 1}, Total Reward: {total_reward}")

In [1]:
import torch
import torch.nn as nn

# 단어 사전 크기와 임베딩 차원 정의
vocab_size = 10000  # 단어 사전 크기
embedding_dim = 300  # 임베딩 차원

# 임베딩 레이어 생성
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# 예제 입력 (단어 인덱스)
input_indices = torch.tensor([1, 2, 3, 4])  # 예: 단어 인덱스 시퀀스

# 임베딩 레이어를 통한 입력 변환
embedded_output = embedding_layer(input_indices)

print(f"Embedded output shape: {embedded_output.shape}")  # (시퀀스 길이, 임베딩 차원)
print(f"Embedded output: {embedded_output}")  # 임베딩된 벡터 출력

Embedded output shape: torch.Size([4, 300])
Embedded output: tensor([[-1.4593, -0.2077,  0.7009,  ...,  0.4990, -0.6829, -0.1326],
        [-0.0071,  0.4374,  0.1842,  ..., -0.1883,  0.7059, -2.1017],
        [-0.5948,  0.4116, -1.9910,  ..., -0.0759,  2.3095, -0.7185],
        [-1.1674, -1.1267,  0.9069,  ...,  0.0464,  2.4247, -1.1143]],
       grad_fn=<EmbeddingBackward0>)
